In [71]:
import pandas as pd
from evidently import ColumnMapping
from evidently.report import Report

In [72]:
jan_data = pd.read_parquet('./data/green_tripdata_2022-02.parquet')
print(jan_data.shape)
jan_data.describe()

(69399, 20)


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,69399.000000,69399,69399,61978.000000,69399.000000,69399.000000,61978.00000,69399.000000,69399.000000,69399.000000,69399.000000,69399.000000,69399.000000,69399.000000,69399.000000,61978.000000,61978.000000,61978.000000
mean,1.850517,2022-02-14 22:21:25.209613,2022-02-14 22:41:19.206689,1.198796,98.913904,136.606003,1.26795,96.154986,14.070630,0.356017,0.423729,1.801974,0.202369,0.297471,17.788976,1.391187,1.042386,0.719102
min,1.000000,2009-01-01 00:02:32,2009-01-01 14:31:35,1.000000,1.000000,1.000000,0.00000,0.000000,-165.000000,-4.500000,-0.500000,-0.990000,0.000000,-0.300000,-165.300000,1.000000,1.000000,-2.750000
25%,2.000000,2022-02-08 02:29:01,2022-02-08 03:16:25,1.000000,61.000000,74.000000,1.00000,1.100000,7.500000,0.000000,0.500000,0.000000,0.000000,0.300000,9.780000,1.000000,1.000000,0.000000
50%,2.000000,2022-02-14 23:50:36,2022-02-15 00:08:39,1.000000,75.000000,138.000000,1.00000,1.930000,10.500000,0.000000,0.500000,1.160000,0.000000,0.300000,14.160000,1.000000,1.000000,0.000000
75%,2.000000,2022-02-22 06:54:11.500000,2022-02-22 07:15:26.500000,1.000000,130.000000,215.000000,1.00000,3.560000,17.000000,0.500000,0.500000,2.760000,0.000000,0.300000,21.350000,2.000000,1.000000,2.750000
max,2.000000,2022-03-01 10:04:17,2022-03-01 20:09:43,6.000000,265.000000,265.000000,9.00000,259777.570000,420.000000,4.500000,3.550000,250.000000,38.750000,0.300000,421.300000,5.000000,2.000000,2.750000
std,0.356567,NaN,NaN,0.861625,63.128032,77.125833,0.89894,3335.401400,12.266505,0.661292,0.184683,3.118095,1.186615,0.035359,13.988396,0.507374,0.201470,1.208780


In [73]:
jan_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2022-02-01 00:20:21,2022-02-01 00:24:30,N,1.0,43,238,1.0,1.16,5.5,0.5,0.5,1.02,0.0,None,0.3,7.82,1.0,1.0,0.00
1,2,2022-02-01 00:32:26,2022-02-01 00:35:31,N,1.0,166,24,1.0,0.57,4.5,0.5,0.5,0.00,0.0,None,0.3,5.80,2.0,1.0,0.00
2,1,2022-02-01 00:17:27,2022-02-01 00:44:44,N,1.0,226,219,1.0,0.00,42.2,0.0,0.5,0.00,0.0,None,0.3,43.00,1.0,1.0,0.00
3,2,2022-02-01 00:45:37,2022-02-01 01:27:16,N,1.0,89,83,1.0,16.62,49.0,0.5,0.5,0.00,0.0,None,0.3,50.30,2.0,1.0,0.00
4,2,2022-02-01 00:06:46,2022-02-01 00:30:06,N,1.0,7,238,1.0,5.97,21.0,0.5,0.5,4.50,0.0,None,0.3,29.55,1.0,1.0,2.75


In [74]:
train_data = jan_data[:30000]
val_data = jan_data[30000:]

In [75]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]


In [79]:
val_data.loc[val_data.lpep_pickup_datetime.between('2022-01-28', '2022-01-29')]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
42807,2,2022-01-28 12:40:06,2022-01-28 12:44:10,N,1.0,75,75,1.0,0.41,4.5,0.5,0.5,0.0,0.0,None,0.3,5.8,2.0,1.0,0.0
42859,2,2022-01-28 15:03:25,2022-01-28 15:19:44,N,1.0,7,138,1.0,4.07,15.5,0.0,0.5,0.0,0.0,None,0.3,16.3,2.0,1.0,0.0


### Evidently Report Project

In [80]:
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features
)

In [81]:
import datetime
from evidently.ui.workspace import Workspace
from evidently.renderers.html_widgets import WidgetSize
from evidently.metric_preset import DataDriftPreset, DataQualityPreset
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter


In [82]:
ws = Workspace("workspace")

In [83]:
project = ws.create_project("NYC Taxi Data Quality Project")
project.description = "My project descriotion"
project.save()

Project(id=UUID('01925dec-f4a8-76d0-afd0-a2df425e72b5'), name='NYC Taxi Data Quality Project', description='My project descriotion', dashboard=DashboardConfig(name='NYC Taxi Data Quality Project', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 10, 6, 1, 24, 23, 208199))

In [84]:
regular_report = Report(
    metrics=[
        DataQualityPreset()
    ],
    timestamp=datetime.datetime(2022,1,28)
)

regular_report.run(reference_data=None,
                  current_data=val_data.loc[val_data.lpep_pickup_datetime.between('2022-01-28', '2022-01-29', inclusive="left")],
                  column_mapping=column_mapping)


In [85]:
ws.add_report(project.id, regular_report)

In [86]:
#configure the dashboard
project.dashboard.add_panel(
    DashboardPanelCounter(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        agg=CounterAgg.NONE,
        title="NYC taxi data dashboard"
    )
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Inference Count",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_rows",
                legend="count"
            ),
        ],
        plot_type=PlotType.BAR,
        size=WidgetSize.HALF,
    ),
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Number of Missing Values",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_missing_values",
                legend="count"
            ),
        ],
        plot_type=PlotType.LINE,
        size=WidgetSize.HALF,
    ),
)

project.save()

Project(id=UUID('01925dec-f4a8-76d0-afd0-a2df425e72b5'), name='NYC Taxi Data Quality Project', description='My project descriotion', dashboard=DashboardConfig(name='NYC Taxi Data Quality Project', panels=[DashboardPanelCounter(type='evidently:dashboard_panel:DashboardPanelCounter', id=UUID('01925dec-f6dc-7c6f-ad7c-b5a86af9b5b7'), title='NYC taxi data dashboard', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.FULL: 2>, agg=<CounterAgg.NONE: 'none'>, value=None, text=None), DashboardPanelPlot(type='evidently:dashboard_panel:DashboardPanelPlot', id=UUID('01925dec-f6dd-726f-861e-742b15b230a5'), title='Inference Count', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.HALF: 1>, values=[PanelValue(field_path='current.number_of_rows', metric_id='DatasetSummaryMetric', metric_fingerprint=None, metric_args={}, legend='count')], plot_type=<PlotType.BAR: 'bar'>), DashboardPanelPlot(type='eviden

In [87]:
val_data.loc[val_data.lpep_pickup_datetime.between('2022-01-29', '2022-01-30')].isna()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
44468,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [88]:
regular_report = Report(
    metrics=[
        DataQualityPreset()
    ],
    timestamp=datetime.datetime(2022,1,29)
)

regular_report.run(reference_data=None,
                  current_data=val_data.loc[val_data.lpep_pickup_datetime.between('2022-01-29', '2022-01-30', inclusive="left")],
                  column_mapping=column_mapping)

ws.add_report(project.id, regular_report)

/opt/anaconda3/envs/dev_env/lib/python3.10/site-packages/evidently/metrics/data_integrity/column_summary_metric.py:206: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'Y' instead.
  curr_data = current_data.dt.to_period(freq=freq).value_counts().reset_index()
/opt/anaconda3/envs/dev_env/lib/python3.10/site-packages/evidently/metrics/data_integrity/column_summary_metric.py:206: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'Y' instead.
  curr_data = current_data.dt.to_period(freq=freq).value_counts().reset_index()
